#  第2章 Rによるデータハンドリングを学びたい <br>---アンケートデータとID-POSデータのハンドリング

目次を見た感じだと1章と比較してRの手法メインの章かなという印象

## 手法の概要

予想という感じ。pandasで似たようなことをやっていこう。

dataframeでの対応をまとめる
str | dtypes

In [23]:
import pandas as pd
import numpy as np

## 変数の型

### 関数strによるデータ構造の把握

In [24]:
df_jhk = pd.read_csv("../data/第02章/人事評価結果.csv", encoding="shift-jis")

In [25]:
# df_jhk[:5]

In [26]:
df_jhk.dtypes

ID       int64
性別      object
部署      object
年代      object
協調性      int64
自己主張     int64
技能       int64
知識       int64
ストレス     int64
総合       int64
昨年総合     int64
dtype: object

python では、strがobjectになっている。\
Rに関して言うと、Factorは嫌いというか使えない。

In [27]:
df_jhk["総合平均"] = (df_jhk["昨年総合"] + df_jhk["総合"])/2

In [28]:
df_jhk.dtypes

ID        int64
性別       object
部署       object
年代       object
協調性       int64
自己主張      int64
技能        int64
知識        int64
ストレス      int64
総合        int64
昨年総合      int64
総合平均    float64
dtype: object

総合平均がfloatで足されたのがわかる。

### 関数factorの使いどころ

In [29]:
S = pd.Series([1,5,2,10,8,2,1,4,3,3])

すこし強引にやる

In [30]:
pd.cut(S, range(-1,10)).value_counts(sort = False)

(-1, 0]    0
(0, 1]     2
(1, 2]     2
(2, 3]     2
(3, 4]     1
(4, 5]     1
(5, 6]     0
(6, 7]     0
(7, 8]     1
(8, 9]     0
dtype: int64

## 観測対象の情報の抽出

queryを使えるとかなり便利にDataFrameを使用できる。

In [31]:
mdat = df_jhk.query("性別 == 'M'")
# mdat.head()

In [32]:
mdat2 = df_jhk.query("性別 == 'F'")
# mdat.head()

いろいろな方法でやってみる

In [33]:
cope1 = df_jhk.query("協調性 < 50") # < 50
cope2 = df_jhk.where( df_jhk["協調性"] <= 50).dropna()# <= 50

cope3 = df_jhk[df_jhk["協調性"] > 50]# > 50
cope4 = df_jhk.query("協調性 >= 50") # >=50

In [34]:
max(cope2["協調性"])

50.0

ちなみにこんなこともできる

In [35]:
cope5 = df_jhk.query("75 > 協調性 > 50")

In [36]:
min(cope5["協調性"]), max(cope5["協調性"])

(51, 74)

In [37]:
m1 = df_jhk.query("(性別 == 'M') | (年代 == '熟練')") # M and 熟練
m2 = df_jhk.where( (df_jhk["技能"] >= 50) & (df_jhk["性別"] == "M") & (df_jhk["年代"] == "熟練")).dropna()# 

cope3 = df_jhk[(df_jhk["技能"] >= 50) & (df_jhk["性別"] == "M") | (df_jhk["年代"] == "熟練")]

In [38]:
# m2

## 欠損値の処理

In [39]:
df_na = pd.read_csv("../data/第02章/欠損データ.csv")

In [40]:
df_na

,ID,A,B,C,D
0,1,5.0,31,95,23.0
1,2,NaN,23,73,33.0
2,3,23.0,35,43,54.0
3,4,3.0,45,8,NaN


In [41]:
df_na2 = pd.read_csv("../data/第02章/欠損データ2.csv")

In [42]:
df_na2

,ID,A,B,C,D
0,1,5,31,95,23
1,2,999,23,73,33
2,3,23,35,43,54
3,4,3,45,8,9999


Pythonには、na.stringsのようなものはなさそう。\
ただ条件に合うものを埋めるのは、where!\
実際は、合わないものを埋めるので、条件は、999以下

In [43]:
kesson = df_na2.where( ( 999 > df_na2), other = np.nan )
kesson

,ID,A,B,C,D
0,1,5.0,31,95,23.0
1,2,NaN,23,73,33.0
2,3,23.0,35,43,54.0
3,4,3.0,45,8,NaN


削除は、dropna

In [44]:
kanzen = kesson.dropna(axis=0)
kanzen

,ID,A,B,C,D
0,1,5.0,31,95,23.0
2,3,23.0,35,43,54.0


In [45]:
df_na.isnull().any(axis=0)

ID    False
A      True
B     False
C     False
D      True
dtype: bool

### 一回欠損値の確認の方法(Python用番外編)

In [46]:
df_na.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4 entries, 0 to 3
Data columns (total 5 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   ID      4 non-null      int64  
 1   A       3 non-null      float64
 2   B       4 non-null      int64  
 3   C       4 non-null      int64  
 4   D       3 non-null      float64
dtypes: float64(2), int64(3)
memory usage: 288.0 bytes


In [47]:
df_na.isnull()

,ID,A,B,C,D
0,False,False,False,False,False
1,False,True,False,False,False
2,False,False,False,False,False
3,False,False,False,False,True


In [48]:
df_na.isnull().any(axis = 0)

ID    False
A      True
B     False
C     False
D      True
dtype: bool

## ソート

In [49]:
score = [1,5,2,10,8,2,1,4,3,3]

In [50]:
sorted(score)

[1, 1, 2, 2, 3, 3, 4, 5, 8, 10]

In [51]:
df_s = pd.read_csv("../data/第02章/ソートデータ.csv", encoding="shift-jis")

In [52]:
df_s

,部署,協調性,ストレス,総合
0,1,30,40,10
1,3,25,10,50
2,2,50,20,20
3,1,40,40,30
4,2,20,30,40
5,2,10,20,40
6,3,20,10,30


In [53]:
df_s.sort_values(by = "協調性")

,部署,協調性,ストレス,総合
5,2,10,20,40
4,2,20,30,40
6,3,20,10,30
1,3,25,10,50
0,1,30,40,10
3,1,40,40,30
2,2,50,20,20


indexがpythonは0、Rは1始まりなので、少しずれたように見える

In [54]:
df_s.sort_values(by = ["協調性", "総合"])

,部署,協調性,ストレス,総合
5,2,10,20,40
6,3,20,10,30
4,2,20,30,40
1,3,25,10,50
0,1,30,40,10
3,1,40,40,30
2,2,50,20,20


## マージ

In [55]:
datA = pd.read_csv("../data/第02章/マージデータA.csv", encoding="shift-jis")
datB = pd.read_csv("../data/第02章/マージデータB.csv", encoding="shift-jis")

In [56]:
datA

,ID,質問A,質問B
0,1,3,1
1,2,3,5
2,3,2,6
3,4,0,7
4,5,1,3
5,6,4,3
6,7,4,7
7,8,4,3


In [57]:
datB

,ID,質問C,質問D
0,1,30,100
1,3,20,60
2,5,10,30
3,7,40,70


In [58]:
pd.merge(datA, datB, on = "ID", how = "left")

,ID,質問A,質問B,質問C,質問D
0,1,3,1,30.0,100.0
1,2,3,5,NaN,NaN
2,3,2,6,20.0,60.0
3,4,0,7,NaN,NaN
4,5,1,3,10.0,30.0
5,6,4,3,NaN,NaN
6,7,4,7,40.0,70.0
7,8,4,3,NaN,NaN


## 数値の置き換え

In [59]:
vec = [2,3,4,5,1,2,3,1,2]
mat = np.array(vec).reshape(3,-1).T

.Tは転置

In [60]:
mat

array([[2, 5, 3],
       [3, 1, 1],
       [4, 2, 2]])

pythonは0始まりなので、Rの結果引く1になっている

In [61]:
loc2 = np.where(mat == 2)
loc2

(array([0, 2, 2], dtype=int64), array([0, 1, 2], dtype=int64))

In [62]:
loc4 = np.where(mat == 4)

In [63]:
mat[loc2] = 4
mat[loc4] = 2

In [64]:
mat

array([[4, 5, 3],
       [3, 1, 1],
       [2, 4, 4]])

## 固定長データのハンドリング

テキストデータの読み込み

In [65]:
with open("../data/第02章/項目反応固定長.txt") as f:
    itermap = f.readlines()

In [66]:
itermap

['ID000142414\n',
 'ID000211111\n',
 'ID000314144\n',
 'ID000434413\n',
 'ID000532112\n']

改行コード(\n)も残ってしまうので、replaceする

In [67]:
itermap = [i.replace('\n','') for i in itermap]

In [68]:
itermap

['ID000142414', 'ID000211111', 'ID000314144', 'ID000434413', 'ID000532112']

In [69]:
row0 = [[i[:6],i[6],i[7], i[8], i[9], i[10]] for i in itermap]

少し強引か

In [70]:
row0

[['ID0001', '4', '2', '4', '1', '4'],
 ['ID0002', '1', '1', '1', '1', '1'],
 ['ID0003', '1', '4', '1', '4', '4'],
 ['ID0004', '3', '4', '4', '1', '3'],
 ['ID0005', '3', '2', '1', '1', '2']]

*はリストを展開する

In [71]:
df_row = pd.DataFrame(row0, columns=["ID",*[f"問{i+1}"for i in range(5)] ])

In [72]:
df_row

,ID,問1,問2,問3,問4,問5
0,ID0001,4,2,4,1,4
1,ID0002,1,1,1,1,1
2,ID0003,1,4,1,4,4
3,ID0004,3,4,4,1,3
4,ID0005,3,2,1,1,2


In [73]:
with open("../data/第02章/key.txt") as f:
    answer = f.readlines()

In [74]:
answer

['正答\n', '2\n', '1\n', '4\n', '1\n', '1\n']

In [75]:
answer = [a.replace("\n", "") for a in answer]

In [76]:
key = answer[1:]

In [77]:
key_tile = np.tile(key, (len(df_row), 1 ))

In [78]:
key_tile

array([['2', '1', '4', '1', '1'],
       ['2', '1', '4', '1', '1'],
       ['2', '1', '4', '1', '1'],
       ['2', '1', '4', '1', '1'],
       ['2', '1', '4', '1', '1']], dtype='<U1')

pythonは特殊で、True,Falseは元がintである。\
少し雑な言い方（笑）

In [79]:
(df_row.iloc[:, 1:] == key_tile).astype(int)

,問1,問2,問3,問4,問5
0,0,0,1,1,0
1,0,1,0,1,1
2,0,0,0,0,0
3,0,0,1,1,0
4,0,0,0,1,0


pythonでboolenがintといったおもしろい例

In [80]:
True + True

2

In [81]:
True*4

4

なんてこともできる

## ID-POSデータの読み込み

In [82]:
import glob

In [83]:
fname = glob.glob("../data/第02章/POSフォルダ/*")

In [84]:
fname

['../data/第02章/POSフォルダ\\201301.csv',
 '../data/第02章/POSフォルダ\\201302.csv',
 '../data/第02章/POSフォルダ\\201303.csv',
 '../data/第02章/POSフォルダ\\201304.csv',
 '../data/第02章/POSフォルダ\\201305.csv',
 '../data/第02章/POSフォルダ\\201306.csv',
 '../data/第02章/POSフォルダ\\201307.csv',
 '../data/第02章/POSフォルダ\\201308.csv',
 '../data/第02章/POSフォルダ\\201309.csv',
 '../data/第02章/POSフォルダ\\201310.csv',
 '../data/第02章/POSフォルダ\\201311.csv',
 '../data/第02章/POSフォルダ\\201312.csv']

In [85]:
pos0 = pd.read_csv(fname[0], encoding="shift-jis")

In [86]:
pos0[:5]

,顧客ID,店舗,購買日,購買時間,商品カテゴリ,購買金額
0,ID00001,B,20130108,15.60,C4,322
1,ID00002,B,20130101,17.60,C25,320
2,ID00002,B,20130110,17.71,C1,383
3,ID00002,B,20130131,15.65,C25,222
4,ID00003,C,20130102,17.25,C14,365


### コラム　Rの外部エディタとしての"NotePad"
個人的に、Notepadは使ったことないが、RStadioはかなり使いやすい。データ解析をする授業で、レポートを書きながら実行して、そのままwordに変換できたのでだれよりも早く終われたので大好き。

In [87]:
pos0.dtypes

顧客ID       object
店舗         object
購買日         int64
購買時間      float64
商品カテゴリ     object
購買金額        int64
dtype: object

In [88]:
list_df = [pd.read_csv(f, encoding="shift-jis") for f in fname]

In [89]:
posall = pd.DataFrame()
for df_one  in list_df:
    posall = pd.concat([posall, df_one])

In [90]:
posall.describe()

,購買日,購買時間,購買金額
count,1.000000e+03,1000.000000,1000.000000
mean,2.013067e+07,15.929680,301.424000
std,3.519459e+02,1.512332,81.359139
min,2.013010e+07,9.300000,4.000000
25%,2.013033e+07,14.930000,247.750000
50%,2.013071e+07,15.960000,303.000000
75%,2.013100e+07,16.942500,358.000000
max,2.013123e+07,20.950000,604.000000


## ID-POSデータにおけるソート

In [91]:
posall.sort_values(by = ["顧客ID", "購買日", "購買時間"])[:6]

,顧客ID,店舗,購買日,購買時間,商品カテゴリ,購買金額
0,ID00001,B,20130108,15.60,C4,322
0,ID00001,B,20130202,14.61,C28,332
1,ID00001,A,20130204,18.29,C1,254
2,ID00001,B,20130219,17.14,C27,385
0,ID00001,C,20130320,17.39,C1,406
0,ID00001,C,20130407,18.38,C30,289


## RFM分析

In [92]:
R = pd.pivot_table(posall, index="顧客ID", values = "購買日", aggfunc=np.max)

In [93]:
F = posall["顧客ID"].value_counts()

In [94]:
M =  pd.pivot_table(posall, index="顧客ID", values = "購買金額", aggfunc=np.sum)

In [95]:
rfm = pd.concat([R, F, M], axis = 1)

In [96]:
rfm.columns = ["R", "F","M"]

In [97]:
rfm[:5]

,R,F,M
ID00001,20131215,23,7026
ID00002,20131231,37,11436
ID00003,20131228,23,7303
ID00004,20131229,29,9371
ID00005,20131230,33,10743


In [98]:
rfm2 = rfm.sort_values(by=["R", "F", "M"], ascending=False)
rfm2[:7]

,R,F,M
ID00002,20131231,37,11436
ID00020,20131231,35,11045
ID00010,20131231,32,9862
ID00025,20131231,32,9608
ID00024,20131231,26,7897
ID00035,20131231,23,7088
ID00005,20131230,33,10743


## ID-POSデータにおけるクロス集計表

In [99]:
t1 = pd.crosstab(posall["顧客ID"], posall["商品カテゴリ"])

In [100]:
t1[:5]

商品カテゴリ,C1,C10,C11,C12,C13,C14,C15,C16,C17,C18,...,C28,C29,C3,C30,C4,C5,C6,C7,C8,C9
顧客ID,,,,,,,,,,,,,,,,,,,,,
ID00001,2,0,0,1,0,2,1,0,0,1,...,2,1,1,2,2,0,0,0,0,0
ID00002,2,0,0,0,1,1,1,2,0,2,...,1,2,1,1,2,0,2,2,0,1
ID00003,0,2,1,0,1,2,0,0,0,3,...,0,3,0,1,0,1,0,0,1,1
ID00004,0,0,2,1,0,1,1,1,2,1,...,0,2,4,2,0,0,1,1,0,3
ID00005,0,4,2,0,2,2,0,0,1,1,...,0,0,2,2,0,0,1,2,2,1


In [101]:
t1 = pd.crosstab(posall["顧客ID"], posall["商品カテゴリ"])

In [102]:
# t1

In [103]:
t2 = pd.crosstab([posall["商品カテゴリ"],posall["店舗"]], columns=posall["顧客ID"])

In [104]:
# t2

In [105]:
t3 = pd.crosstab([posall["商品カテゴリ"],posall["購買日"]], columns=posall["顧客ID"])

In [106]:
t3

顧客ID             ID00001  ID00002  ID00003  ID00004  ID00005  ID00006  \
商品カテゴリ 購買日                                                              
C1     20130110        0        1        0        0        0        0   
       20130121        0        0        0        0        0        0   
       20130204        1        0        0        0        0        0   
       20130205        0        0        0        0        0        0   
       20130210        0        0        0        0        0        0   
...                  ...      ...      ...      ...      ...      ...   
C9     20131117        0        0        0        0        0        0   
       20131125        0        0        0        0        0        0   
       20131205        0        0        0        0        0        0   
       20131225        0        0        0        0        0        0   
       20131231        0        0        0        0        0        0   

顧客ID             ID00007  ID00008  ID00009  ID00010  ...  ID00027  ID00028  \
商品カテゴリ 購買日                                           ...                     
C1     20130110        0        0        0        0  ...        0        0   
       20130121        0        0        0        0  ...        0        0   
       20130204        0        0        0        0  ...        0        0   
       20130205        0        0        0        0  ...        0        0   
       20130210        0        0        0        0  ...        0        0   
...                  ...      ...      ...      ...  ...      ...      ...   
C9     20131117        0        0        0        0  ...        0        0   
       20131125        0        0        0        0  ...        0        1   
       20131205        0        0        0        0  ...        0        0   
       20131225        0        1        0        0  ...        0        0   
       20131231        0        0        0        0  ...        0        0   

顧客ID             ID00029  ID00030  ID00031  ID00032  ID00033  ID00034  \
商品カテゴリ 購買日                                                              
C1     20130110        0        0        0        0        0        0   
       20130121        0        0        0        0        0        0   
       20130204        0        0        0        0        0        0   
       20130205        0        0        0        0        0        0   
       20130210        0        0        0        0        0        0   
...                  ...      ...      ...      ...      ...      ...   
C9     20131117        0        0        0        0        0        0   
       20131125        0        0        0        0        0        0   
       20131205        0        0        0        0        0        0   
       20131225        0        0        0        0        0        0   
       20131231        0        0        0        0        0        0   

顧客ID             ID00035  ID00036  
商品カテゴリ 購買日                         
C1     20130110        0        0  
       20130121        0        0  
       20130204        0        0  
       20130205        0        0  
       20130210        0        0  
...                  ...      ...  
C9     20131117        0        0  
       20131125        0        0  
       20131205        0        0  
       20131225        0        0  
       20131231        1        0  

[951 rows x 36 columns]

In [107]:
t1.shape

(36, 30)

In [108]:
storeA = posall.query("店舗 == 'A'")

In [109]:
pd.crosstab(storeA["顧客ID"],storeA["商品カテゴリ"]).shape

(35, 30)

35行になっているので見てみると、ID00036が店舗Aの値が0になっている。\
queryだとデータがないと削除されてしまうで、クロス集計表を作ってそのなかで顧客IDと商品カテゴリをとる

In [110]:
cross = pd.crosstab(posall["顧客ID"],[posall["商品カテゴリ"], posall["店舗"]])

columnsが二段階になっているmultiindexになっているので、少し特殊なやり方、slice(None)は一階層目をすべて選択することになっている。

In [111]:
cross.loc[:, (slice(None), 'A')].shape

(36, 30)

## 顧客ID 別に月ごとの購買金額を求める

In [112]:
cid = posall.顧客ID

In [113]:
posall.columns

Index(['顧客ID', '店舗', '購買日', '購買時間', '商品カテゴリ', '購買金額'], dtype='object')

In [114]:
buym = posall.購買日.map(lambda x : f"{str(x)[:6]}" )

In [115]:
pd.pivot_table(posall, "購買金額",[cid,buym], np.sum)

0      1      2   3   4   5   6   7   8   9   ...     86  \
顧客ID    購買日                                                      ...          
ID00001 201301  322.0    NaN    NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
        201302  332.0  254.0  385.0 NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
        201303  406.0    NaN    NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
        201304  289.0  293.0  122.0 NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
        201305  291.0  265.0    NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
...               ...    ...    ...  ..  ..  ..  ..  ..  ..  ..  ...    ...   
ID00036 201306    NaN    NaN    NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
        201307    NaN    NaN    NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
        201308    NaN    NaN    NaN NaN NaN NaN NaN NaN NaN NaN  ...  317.0   
        201309    NaN    NaN    NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   
        201310    NaN    NaN    NaN NaN NaN NaN NaN NaN NaN NaN  ...    NaN   

                   87  88  89  90     91     92     93  94  95  
顧客ID    購買日                                                     
ID00001 201301    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  
        201302    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  
        201303    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  
        201304    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  
        201305    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  
...               ...  ..  ..  ..    ...    ...    ...  ..  ..  
ID00036 201306    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  
        201307    NaN NaN NaN NaN  346.0  196.0  317.0 NaN NaN  
        201308  228.0 NaN NaN NaN    NaN    NaN    NaN NaN NaN  
        201309    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  
        201310    NaN NaN NaN NaN    NaN    NaN    NaN NaN NaN  

[397 rows x 96 columns]

いい方法がわからなかったが、日当たりの合計になってしまった。\
月ごとにする

In [116]:
pd.pivot_table(posall, "購買金額",[cid,buym], np.sum).sum(axis=1)

顧客ID     購買日   
ID00001  201301     322.0
         201302     971.0
         201303     406.0
         201304     704.0
         201305     556.0
                    ...  
ID00036  201306    1135.0
         201307     859.0
         201308     952.0
         201309     284.0
         201310     681.0
Length: 397, dtype: float64

## 顧客ID 別に商品名を取得する---自作関数を利用する

for t in t1

In [117]:
res2 = t1.apply(lambda x : t1.columns[x > 1], axis=1)

ソートのされ方が異なっているので表示が若干異なる

In [118]:
res2.head(2)

顧客ID
ID00001    Index(['C1', 'C14', 'C20', 'C22', 'C28', 'C30'...
ID00002    Index(['C1', 'C16', 'C18', 'C19', 'C22', 'C24'...
dtype: object

## 顧客ID ごとに来店感覚の分布を描画・要約する

In [119]:
from datetime import datetime

In [120]:
posall["ndate"] = posall["購買日"].map(lambda date : datetime((date)//10000, (date - (20130108//10000)*10000)//100,((date)%100)))

なんだか、numpyの日付の型は、計算するとなの秒になるらしい 

In [164]:
posall.query("顧客ID == 'ID00001'")["ndate"][:5]

0   2013-01-08
0   2013-02-02
1   2013-02-04
2   2013-02-19
0   2013-03-20
Name: ndate, dtype: datetime64[ns]

In [174]:
np.diff(posall.query("顧客ID == 'ID00001'")["ndate"])[:5]

array([2160000000000000,  172800000000000, 1296000000000000,
       2505600000000000, 1555200000000000], dtype='timedelta64[ns]')

In [176]:
np.diff(posall.query("顧客ID == 'ID00001'")["ndate"])/(3600*10**9*24)

array([25,  2, 15, 29, 18,  3, 12, 10, 18, 22,  2, 22, 37, 14,  5, 13,  3,
        0, 27, 12, 26, 26], dtype='timedelta64[ns]')

In [260]:
restime =  [ (np.diff(posall.query("顧客ID == 'ID000{:02}'".format(i))["ndate"])/(3600*10**9*24)).astype(int) for i in range(len(posall.顧客ID.unique())+1)][1:]

In [261]:
len(restime)

36

In [262]:
df_restime = pd.DataFrame(restime, index = posall.顧客ID.unique())

In [268]:
df_restime[:5]

,0,1,2,3,4,5,6,7,8,9,...,27,28,29,30,31,32,33,34,35,36
ID00001,25,2,15,29,18,3,12,10,18,22,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID00002,9,21,26,2,7,8,18,11,1,1,...,6.0,9.0,8.0,4.0,24.0,12.0,15.0,2.0,11.0,NaN
ID00003,9,1,7,9,23,12,76,1,21,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID00004,2,12,1,6,5,1,13,8,29,49,...,16.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
ID00005,14,2,8,3,5,14,24,2,6,1,...,9.0,11.0,4.0,6.0,26.0,NaN,NaN,NaN,NaN,NaN


In [274]:
df_restime.index.name = "顧客ID"

dataframeのdescribeは列方向しかできないので

In [345]:
df_restime.T.describe()

顧客ID,ID00001,ID00002,ID00003,ID00004,ID00005,ID00006,ID00007,ID00008,ID00009,ID00010,...,ID00030,ID00031,ID00032,ID00033,ID00034,ID00035,ID00036,ID00021,ID00025,ID00029
count,22.000000,36.000000,22.000000,28.000000,32.000000,27.000000,19.000000,25.00000,19.000000,31.000000,...,31.000000,33.000000,29.000000,33.000000,22.000000,28.000000,19.000000,16.00000,22.000000,16.000000
mean,15.500000,10.111111,16.363636,12.142857,10.531250,13.037037,18.894737,13.40000,17.578947,11.483871,...,10.967742,10.696970,11.344828,10.181818,15.363636,12.071429,18.263158,21.31250,16.409091,17.875000
std,10.409474,8.331619,18.256233,13.707836,9.030359,16.577282,19.561569,9.69536,16.523773,8.850126,...,10.219210,7.630059,10.434407,13.687254,15.558019,9.172942,14.737865,18.56419,16.692340,22.144601
min,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.00000,0.000000,1.000000,...,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.00000,0.000000,0.000000
25%,6.250000,3.750000,7.000000,2.000000,4.000000,4.000000,5.500000,9.00000,4.000000,4.000000,...,3.500000,5.000000,3.000000,4.000000,5.250000,4.750000,5.500000,6.75000,4.250000,4.750000
50%,14.500000,8.500000,11.000000,6.500000,9.000000,9.000000,14.000000,10.00000,15.000000,10.000000,...,9.000000,8.000000,9.000000,6.000000,12.500000,10.000000,13.000000,14.50000,10.000000,11.500000
75%,24.250000,14.250000,19.500000,16.000000,14.000000,14.000000,25.500000,18.00000,21.500000,16.000000,...,13.500000,16.000000,16.000000,13.000000,19.000000,17.250000,32.500000,26.50000,21.750000,23.500000
max,37.000000,31.000000,76.000000,51.000000,37.000000,84.000000,84.000000,42.00000,55.000000,34.000000,...,40.000000,34.000000,42.000000,73.000000,70.000000,41.000000,44.000000,57.00000,63.000000,91.000000
